In [1]:
import numpy as np
import pandas as pd 
from BHDVCS_tf import BHDVCStf
from BHDVCS_tf import TotalFLayer
import utilities as uts
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt

import sys
from scipy.stats import chisquare

tb = BHDVCStf()
f = tb.curve_fit
f_arr = tb.curve_fit_array

df = pd.read_csv("BKM_pseudodata.csv")
df = df.rename(columns={"sigmaF": "errF"})

data = uts.DvcsData(df)

def F2VsPhi(dataframe,SetNum,xdat,cffs):
    TempFvalSilces=dataframe[dataframe["#Set"]==SetNum]
    TempFvals=TempFvalSilces["F"]
    TempFvals_sigma=TempFvalSilces["errF"]
    temp_phi=TempFvalSilces["phi_x"]
    plt.errorbar(temp_phi,TempFvals,TempFvals_sigma,fmt='.',color='blue',label="Data")
    plt.xlim(0,368)
    temp_unit=(np.max(TempFvals)-np.min(TempFvals))/len(TempFvals)
    plt.ylim(np.min(TempFvals)-temp_unit,np.max(TempFvals)+temp_unit)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.legend(loc=4,fontsize=10,handlelength=3)
    plt.title("Local fit with data set #"+str(SetNum),fontsize=20)
    plt.plot(temp_phi, f(xdat,cffs), 'g--', label='fit')
    file_name = "plot_set_number_{}.png".format(SetNum)
    plt.savefig(file_name)

In [2]:
dats = pd.read_csv('BKM_pseudodata.csv')

k = np.array(dats['k'])
qq = np.array(dats['QQ'])
xb = np.array(dats['x_b'])
t = np.array(dats['t'])
min_t = -1.*t
phi = np.array(dats['phi_x'])
F = np.array(dats['F'])
errF = np.array(dats['sigmaF'])
F1 = np.array(dats['F1']) 
F2 = np.array(dats['F2'])
const = np.array(dats['dvcs'])

i = 0 # datset
a = 45*i # start index of set
b = a+45 # end index of set

#Counterintuitive to change ordering from input file
xdat = np.array([phi[a:b], qq[a:b], xb[a:b], t[a:b], k[a:b], F1[a:b], F2[a:b]])
print(xdat[:, 0])

setI = data.getSet(i, itemsInSet=45)
noCFF = setI.XnoCFF
new_xdat = np.transpose(noCFF.to_numpy())
print(new_xdat[:, 0])

f(new_xdat, [15,10,3,-1])

[ 8.        1.05443   0.536279 -0.452256  2.75      0.439958  0.597371]
[ 8.        1.05443   0.536279 -0.452256  2.75      0.439958  0.597371]


<tf.Tensor: shape=(45,), dtype=float64, numpy=
array([ 1.77045181,  1.44749439,  1.01592314,  0.56842031,  0.16492966,
       -0.16937063, -0.43219975, -0.63220685, -0.78136041, -0.89121256,
       -0.97150171, -1.02990343, -1.07224755, -1.10286898, -1.12495135,
       -1.14081535, -1.15214352, -1.16014871, -1.16569693, -1.16939516,
       -1.17165252, -1.17272112, -1.17272112, -1.17165252, -1.16939516,
       -1.16569693, -1.16014871, -1.15214352, -1.14081535, -1.12495135,
       -1.10286898, -1.07224755, -1.02990343, -0.97150171, -0.89121256,
       -0.78136041, -0.63220685, -0.43219975, -0.16937063,  0.16492966,
        0.56842031,  1.01592314,  1.44749439,  1.77045181,  1.89157839])>

In [3]:
kinematics = tf.keras.Input(shape=(4))
x1 = tf.keras.layers.Dense(100, activation="tanh")(kinematics)
x2 = tf.keras.layers.Dense(100, activation="tanh")(x1)
outputs = tf.keras.layers.Dense(4, activation="linear")(x2)
noncffInputs = tf.keras.Input(shape=(7))
#### phi, kin1, kin2, kin3, kin4, F1, F2 ####
total_FInputs = tf.keras.layers.concatenate([noncffInputs,outputs])
TotalF = TotalFLayer()(total_FInputs)

tfModel = tf.keras.Model(inputs=[kinematics, noncffInputs], outputs = TotalF, name="tfmodel")
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0000005, patience=25)

tfModel.compile(
    optimizer = tf.keras.optimizers.Adam(.0085),
    loss = tf.keras.losses.MeanSquaredError()
)

Wsave = tfModel.get_weights()  

In [4]:
#!!High-overfitting from batch_size 1, 2 100 node hidden layers no validation data, huge number of epochs!!#
# Over-fitting to F will likely not reflect well in CFF predictions

#Number of kinematic sets
by_set = []
for i in range(5):
  setI = data.getSet(i, itemsInSet=45)

  tfModel.set_weights(Wsave)

  tfModel.fit([setI.Kinematics, setI.XnoCFF], setI.sampleY(), # one replica of samples from F vals
                        epochs=15000, verbose=1, batch_size=1, callbacks=[early_stopping_callback])
  
  cffs = uts.cffs_from_globalModel(tfModel, setI.Kinematics, numHL=2)
  

  by_set.append(cffs)

  new_xdat = np.transpose(setI.XnoCFF.to_numpy()) #NB: Could rewrite BHDVCS curve_fit to not require transposition

  F2VsPhi(df,i+1,new_xdat,cffs)
  plt.clf()

Epoch 1/15000
45/45 [==============================] - 4s 3ms/step - loss: 0.4315
Epoch 2/15000
45/45 [==============================] - 0s 4ms/step - loss: 0.0031
Epoch 3/15000
45/45 [==============================] - 0s 4ms/step - loss: 0.0021
Epoch 4/15000
45/45 [==============================] - 0s 4ms/step - loss: 0.0018
Epoch 5/15000
45/45 [==============================] - 0s 4ms/step - loss: 0.0018
Epoch 6/15000
45/45 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 7/15000
45/45 [==============================] - 0s 4ms/step - loss: 0.0014
Epoch 8/15000
45/45 [==============================] - 0s 4ms/step - loss: 0.0014
Epoch 9/15000
45/45 [==============================] - 0s 4ms/step - loss: 0.0027
Epoch 10/15000
45/45 [==============================] - 0s 4ms/step - loss: 0.0028
Epoch 11/15000
45/45 [==============================] - 0s 4ms/step - loss: 0.0022
Epoch 12/15000
45/45 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 13/1500

<Figure size 432x288 with 0 Axes>

In [5]:
by_set

[array([nan, nan, nan, nan], dtype=float32),
 array([nan, nan, nan, nan], dtype=float32),
 array([nan, nan, nan, nan], dtype=float32),
 array([nan, nan, nan, nan], dtype=float32),
 array([nan, nan, nan, nan], dtype=float32)]